### OCI Data Science - Useful Tips
<details>
<summary><font size="2">Check for Public Internet Access</font></summary>

```python
import requests
response = requests.get("https://oracle.com")
assert response.status_code==200, "Internet connection failed"
```
</details>
<details>
<summary><font size="2">Helpful Documentation </font></summary>
<ul><li><a href="https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm">Data Science Service Documentation</a></li>
<li><a href="https://docs.cloud.oracle.com/iaas/tools/ads-sdk/latest/index.html">ADS documentation</a></li>
</ul>
</details>
<details>
<summary><font size="2">Typical Cell Imports and Settings for ADS</font></summary>

```python
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)

import ads
from ads.dataset.factory import DatasetFactory
from ads.automl.provider import OracleAutoMLProvider
from ads.automl.driver import AutoML
from ads.evaluations.evaluator import ADSEvaluator
from ads.common.data import ADSData
from ads.explanations.explainer import ADSExplainer
from ads.explanations.mlx_global_explainer import MLXGlobalExplainer
from ads.explanations.mlx_local_explainer import MLXLocalExplainer
from ads.catalog.model import ModelCatalog
from ads.common.model_artifact import ModelArtifact
```
</details>
<details>
<summary><font size="2">Useful Environment Variables</font></summary>

```python
import os
print(os.environ["NB_SESSION_COMPARTMENT_OCID"])
print(os.environ["PROJECT_OCID"])
print(os.environ["USER_OCID"])
print(os.environ["TENANCY_OCID"])
print(os.environ["NB_REGION"])
```
</details>

# Train and Deploy Transfomer Model using ADS

** conda: pytorch110_p38_gpu_v1

### Install and upgrade required packages - one time for conda

In [1]:
# Upgrade Oracle ADS to pick up latest features and maintain compatibility with Oracle Cloud Infrastructure.
!pip install --upgrade oracle-ads

  Using cached oracle_ads-2.8.3-py3-none-any.whl (1.4 MB)
  Using cached oci-2.97.0-py3-none-any.whl (20.1 MB)
  Attempting uninstall: oci
    Found existing installation: oci 2.88.0
    Uninstalling oci-2.88.0:
      Successfully uninstalled oci-2.88.0
  Attempting uninstall: oracle-ads
    Found existing installation: oracle-ads 2.6.8
    Uninstalling oracle-ads-2.6.8:
      Successfully uninstalled oracle-ads-2.6.8
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
oci-cli 3.20.1 requires oci==2.88.0, but you have oci 2.97.0 which is incompatible.


In [2]:
!pip install --upgrade transformers

  Using cached transformers-4.27.4-py3-none-any.whl (6.8 MB)
  Using cached huggingface_hub-0.13.3-py3-none-any.whl (199 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.10.1
    Uninstalling huggingface-hub-0.10.1:
      Successfully uninstalled huggingface-hub-0.10.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.24.0
    Uninstalling transformers-4.24.0:
      Successfully uninstalled transformers-4.24.0


In [3]:
!pip install ffmpeg-python 

  Using cached ffmpeg_python-0.2.0-py3-none-any.whl (25 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.9/840.9 kB 40.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for future: filename=future-0.18.3-py3-none-any.whl size=492025 sha256=14eac3712ace7eec00b22c4db31f8324879f55003ae78bdbb0850471ca53c35c
  Stored in directory: /home/datascience/.cache/pip/wheels/a0/0b/ee/e6994fadb42c1354dcccb139b0bf2795271bddfe6253ccdf11
Successfully built future


In [4]:
!pip install git+https://github.com/huggingface/datasets#egg=datasets

  Cloning https://github.com/huggingface/datasets to /tmp/pip-install-pl_4ff3k/datasets_1d0fc172e62f4598919209fbbf201c8d
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/datasets /tmp/pip-install-pl_4ff3k/datasets_1d0fc172e62f4598919209fbbf201c8d
  Resolved https://github.com/huggingface/datasets to commit 0803a006db1c395ac715662cc6079651f77c11ea
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached dill-0.3.6-py3-none-any.whl (110 kB)
  Using cached multiprocess-0.70.14-py38-none-any.whl (132 kB)
  Using cached pyarrow-11.0.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (35.0 MB)
  Using cached responses-0.18.0-py3-none-any.whl (38 kB)
  Using cached xxhash-3.2.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (213 kB)
  Created wheel for datasets: filename=datasets-2.11.1.dev0-py3-none-any.whl size=468875 sha256=1522b53956750db0f1d9

In [12]:
!pip install evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  Using cached evaluate-0.4.0-py3-none-any.whl (81 kB)


In [14]:
!pip install rouge_score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  Using cached rouge_score-0.1.2-py3-none-any.whl
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 54.1 MB/s eta 0:00:00


In [6]:
# !pip install --upgrade pyarrow==6.0.0

In [5]:
!pip install --upgrade pyarrow

# Build Transformer model

In [7]:
import pyarrow
print(pyarrow.__version__)

11.0.0


In [8]:
import ads
ads.__version__

'2.8.3'

In [4]:
import os
os.environ["WANDB_DISABLED"] = "true"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [9]:
import transformers
import ads
import time
from datasets import load_dataset
from pathlib import Path
import transformers
from transformers.onnx import FeaturesManager
from transformers import AutoConfig, AutoTokenizer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import pipeline

In [42]:
ads.set_auth("api_key", profile="jiayuan")

In [10]:
billsum = load_dataset("billsum", split="ca_test", download_mode="force_redownload")

Generating train split:   0%|          | 0/18949 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3269 [00:00<?, ? examples/s]

Generating ca_test split:   0%|          | 0/1237 [00:00<?, ? examples/s]

Dataset billsum downloaded and prepared to /home/datascience/.cache/huggingface/datasets/billsum/default/3.0.0/75cf1719d38d6553aa0e0714c393c74579b083ae6e164b2543684e3e92e0c4cc. Subsequent calls will reuse this data.


In [11]:
billsum = billsum.train_test_split(test_size=0.2)

In [13]:
print(billsum["train"][0]['text'])

The people of the State of California do enact as follows:


SECTION 1.
(a) The Legislature finds and declares all of the following:
(1) According to data released by the
U.S.
United States
Census Bureau, without a high school diploma, Americans are almost twice as likely to live in poverty.
(2) Several independent academic studies indicate a marked increase in school participation and graduation rates among children who were guaranteed transportation to and from school.
(3) According to a recent report by
California
Attorney General Kamala Harris, poverty and financial instability is the number one cause of truancy in the state.
(4) Research shows a strong relationship between access to transportation and improved school attendance.
(b) Based on the findings and declarations in subdivision (a), it is the intent of the Legislature to enact legislation that would support school participation and high school attainment among low-income youth.
SEC. 2.
Section 39800 of the Education Code i

In [6]:
print(billsum["train"][0]['summary'])

(1) The Mental Health Services Act (MHSA), an initiative measure enacted by the voters as Proposition 63 at the November 2, 2004, statewide general election, imposes a 1% tax on that portion of a taxpayer’s taxable income that exceeds $1,000,000 and requires that the revenue from that tax be deposited in the Mental Health Services Fund to fund various county mental health programs. The MHSA authorizes the Legislature to amend its provisions by a
2/3
vote, provided that the amendment is consistent with and furthers the intent of the act.
Existing law, known as the No Place Like Home Program, requires the Department of Housing and Community Development to award $2,000,000,000 among counties to finance capital costs, including, but not limited to, acquisition, design, construction, rehabilitation, or preservation, and to capitalize operating reserves, of permanent supportive housing for the target population, as specified. Existing law requires the department to distribute $1,800,000,000 

In [7]:
print(billsum["train"][0]['title'])

An act to add Section 15463 to the Government Code, and to amend Sections 5849.1, 5849.2, 5849.3, 5849.4, 5849.5, 5849.7, 5849.8, 5849.9, 5849.11, 5849.14, 5890, and 5891 of, to add Section 5849.35 to, and to repeal and add Section 5849.13 of, the Welfare and Institutions Code, relating to mental health services, and making an appropriation therefor, to take effect immediately, bill related to the budget.


In [12]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("t5-small")

In [13]:
prefix = "summarize: "


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [14]:
tokenized_billsum = billsum.map(preprocess_function, batched=True)

Map:   0%|          | 0/989 [00:00<?, ? examples/s]

Map:   0%|          | 0/248 [00:00<?, ? examples/s]

In [15]:
import evaluate

rouge = evaluate.load("rouge")

In [16]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

# Train with Pytorch

In [17]:
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [18]:
# define hyperparameter, define output directory of saved model to local, do not push to online hub 
training_args = Seq2SeqTrainingArguments(
    output_dir="pytorch_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True,
    fp16=True,      ##----- only enable if running on gpu
    push_to_hub=False,
    report_to="none"
)

training_args

Seq2SeqTrainingArguments(output_dir='pytorch_model', overwrite_output_dir=False, do_train=False, do_eval=True, do_predict=False, evaluation_strategy=<IntervalStrategy.EPOCH: 'epoch'>, prediction_loss_only=False, per_device_train_batch_size=2, per_device_eval_batch_size=2, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, eval_delay=0, learning_rate=2e-05, weight_decay=0.01, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=4, max_steps=-1, lr_scheduler_type=<SchedulerType.LINEAR: 'linear'>, warmup_ratio=0.0, warmup_steps=0, log_level='passive', log_level_replica='warning', log_on_each_node=True, logging_dir='pytorch_model/runs/May09_16-57-58_bca136a74ae7', logging_strategy=<IntervalStrategy.STEPS: 'steps'>, logging_first_step=False, logging_steps=500, logging_nan_inf_filter=True, save_strategy=<IntervalStrategy.STEPS: 'steps'>, save_steps=500, save_total_limit=3, save_on_each

In [19]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_billsum["train"],
    eval_dataset=tokenized_billsum["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [20]:
# finetune the model

start_time = time.time()
trainer.train()
end_time = time.time()


  warnings.warn(



You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,2.508366,0.178100,0.075100,0.147900,0.148000,19.000000
2,2.924900,2.421140,0.193100,0.091500,0.164300,0.164500,19.000000
3,2.558300,2.382504,0.194300,0.092400,0.164900,0.164900,19.000000
4,2.480000,2.374255,0.194100,0.092000,0.164100,0.164200,19.000000


INFO:absl:Using default tokenizer.
INFO:absl:Using default tokenizer.
INFO:absl:Using default tokenizer.
INFO:absl:Using default tokenizer.


In [21]:
print("--- %s seconds ---" % (end_time - start_time))

--- 360.1333200931549 seconds ---


In [ ]:
tokenized_billsum["test"].head

In [22]:
trainer.predict(tokenized_billsum["test"])

INFO:absl:Using default tokenizer.


PredictionOutput(predictions=array([[    0, 17061,    53, ...,    45,     8, 21104],
       [    0, 17061,    53, ...,     8,   158,  4370],
       [    0, 17061,    53, ...,     3,    99,    34],
       ...,
       [    0, 17061,    53, ...,    12,   136,   568],
       [    0, 17061,    53, ...,    24,     3, 12436],
       [    0, 17061,    53, ...,     8,  6537,    13]]), label_ids=array([[17061,    53,   973, ...,   169,  5161,     1],
       [17061,    53,   973, ...,    15,    26,     1],
       [   37,  1826,  9185, ...,   353,    19,     1],
       ...,
       [ 5637,    37,  1826, ...,   442,   452,     1],
       [17061,    53,   973, ...,  7864,  7173,     1],
       [17061,    53,   973, ...,  2350,    11,     1]]), metrics={'test_loss': 2.3742551803588867, 'test_rouge1': 0.1941, 'test_rouge2': 0.092, 'test_rougeL': 0.1641, 'test_rougeLsum': 0.1642, 'test_gen_len': 19.0, 'test_runtime': 33.7413, 'test_samples_per_second': 7.35, 'test_steps_per_second': 3.675})

In [ ]:
help(trainer)

In [23]:
model_local = AutoModelForSeq2SeqLM.from_pretrained("/home/datascience/Jiayuan/NCH/pytorch_model/checkpoint-1500")
tokenizer_local = AutoTokenizer.from_pretrained("/home/datascience/Jiayuan/NCH/pytorch_model/checkpoint-1500")

In [ ]:
type(tokenizer_local)

# Inference use transformer model

In [30]:
# model.to('cuda')

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [27]:
from transformers import pipeline

summarizer_finetune = pipeline("summarization", model="/home/datascience/Jiayuan/NCH/pytorch_model/checkpoint-1500")
type(summarizer_finetune)

transformers.pipelines.text2text_generation.SummarizationPipeline

In [40]:
## load pipeline using pre-built model

# summarizer = pipeline("summarization", model=model, tokenizer = tokenizer)
# summarizer(text, max_length = 100)

In [25]:
paragraph1 = """If James M. Goodhue could revisit the earth and make a tour among the
daily newspaper offices of St. Paul he would discover that wonderful
strides had been made in the method of producing a newspaper during
the latter half of the past century. Among the first things to attract
the attention of this old-timer would be the web-perfecting press,
capable of producing 25,000 impressions an hour, instead of the old
hand press of 240 impressions an hour; the linotype machine, capable
of setting 6,000 to 10,000 ems per hour, instead of the old hand
compositor producing only 800 to 1,000 ems per hour, and the mailing
machine, enabling one man to do the work of five or six under the
old method. Think of getting out the Sunday Pioneer Press with the
material in use fifty years ago. It would take 600 hand presses, 600
hand pressmen and 600 boys three hours to print the edition, and as
there were no means of stereotyping in those days the forms would have
to be set up 600 times, requiring the services of 5,000 compositors.
Papers printed under these conditions would have to be sold for one
dollar each, and there would not be much profit in it at that. The
first daily papers printed in St. Paul were not conducted or a very
gigantic scale, as the entire force of one office generally consisted
of one pressman, five or six compositors, two editors and a business
manager. A few reminiscences of the trials and tribulations of the
early newspaper manager, editor and compositor may not be wholly
devoid of interest.
"""

summarizer_finetune(paragraph1)

[{'summary_text': 'a newspaper would be produced by a web-perfecting press, capable of producing 25,000 impressions an hour, instead of the old hand press of 240 impressions an hour, and the mailing machine, enabling one man to do the work of five or six under the old method. The Sunday Pioneer Press would take 600 hand presses, 600 hand pressmen and 600 boys three hours to print the edition, and as there were no means of stereotyping in those days the forms would have to be set up 600 times, requiring the services of 25,000 impressions an hour, and the linotype, instead of the old hand press of 240 impressions an hour, and the linotype machine, and the mailing machine, and the linotype machine, and the linotype, and the .'}]

In [39]:
text = """
In 1858, when the first Atlantic cable was laid, the news was
anxiously looked for, and nearly every inhabitant of the city turned
out to greet the arrival of the Gray Eagle and Itasca, two of the
fastest boats on the river, which were expected to bring the news
of the successful laying of the cable. The Gray Eagle started from
Dubuque at 9 o'clock in the morning and the Itasca started from
Prairie du Chien, about 100 miles farther up the river, at noon of the
same day. When the boats reached the bend below the river they were
abreast of each other, and as they reached the levee it was hardly
possible to tell which was ahead. One of the passengers on the Gray
Eagle had a copy of the Dubuque Herald containing the Queen's message,
tied up with a small stone on the inside of it, and as he threw it to
the shore a messenger from the Minnesotian caught it and ran up Bench
street to the Minnesotian office, where the printers were waiting,
and the Minnesotian had the satisfaction of getting out an extra some
little time before their competitors.In the summer season the newspapers had to rely, to a considerable
extent, on the steamboats for late Dubuque and Chicago papers for
telegraph news. There were three or four daily lines of steamers to
St. Paul, and every one of them could be distinguished by its whistle.
When it was time for the arrival of the boat bringing the newspapers
from which the different papers expected to get their telegraphic
news, messengers from the different offices would be at the levee, and
as the boat neared the shore they would leap for the gangplank, and
there was always a scramble to get to the clerk's office first.
James J. Hill and the late Gus Borup were almost always at the levee
awaiting the arrival of the steamers, but as they were after copies
of the boats' manifest they did not come in competition with the
adventurous kids from the newspaper offices."""

summary_text = summarizer_finetune(text, max_length = 50)
summary_text

[{'summary_text': "The Gray Eagle and Itasca were expected to bring the news of the successful laying of the cable, and the itasca started from Dubuque at 9 o'clock in the morning and the Itasca started from Prairie du Chien"}]

In [26]:
text = """
Zoom interface is really simple and easy to conduct virtual meetings. It is very easy to share the Zoom link to join the video conference. And overall screen sharing quality is ok. Zoom now claims to have 300 million meeting participants per day. It chose Oracle Corporation co-founded by Larry Ellison and headquartered in Redwood Shores , for its cloud infrastructure deployments over the likes of Amazon, Microsoft, Google, and even IBM to build an enterprise grade experience for its product.
"""
print(text)
print('summary text: \n', summarizer_finetune(text, max_length = 50))


Zoom interface is really simple and easy to conduct virtual meetings. It is very easy to share the Zoom link to join the video conference. And overall screen sharing quality is ok. Zoom now claims to have 300 million meeting participants per day. It chose Oracle Corporation co-founded by Larry Ellison and headquartered in Redwood Shores , for its cloud infrastructure deployments over the likes of Amazon, Microsoft, Google, and even IBM to build an enterprise grade experience for its product.

summary text: 
 [{'summary_text': 'Zoom now claims to have 300 million meeting participants per day. It chose Oracle Corporation co-founded by Larry Ellison and headquartered in Redwood Shores .'}]


# Prepare Tranformer Model using ADS

In [6]:
from ads.common.model_metadata import UseCaseType
from ads.model import HuggingFacePipelineModel
import tempfile

In [35]:
# Prepare the model

artifact_dir = "t5_model_artifact"
## Initiate a HuggingFacePipelineModel instance
huggingface_pipeline_model = HuggingFacePipelineModel(summarizer_finetune, artifact_dir=artifact_dir)    # model == pipeline
## Prepare
huggingface_pipeline_model.prepare(
    inference_conda_env="oci://conda_env@orasenatdpltintegration03/conda_environments/gpu/PyTorch 1.10 for GPU on Python 3.8/1.0/pytorch110_p38_gpu_v1",      # custom conda
    inference_python_version="3.8",
    training_conda_env="oci://conda_env@orasenatdpltintegration03/conda_environments/gpu/PyTorch 1.10 for GPU on Python 3.8/1.0/pytorch110_p38_gpu_v1",
    use_case_type=UseCaseType.OTHER,
    force_overwrite=True,
    training_script_path="/home/datascience/Jiayuan/NCH/Train&DeployTransformerModel.ipynb"
)

algorithm: SummarizationPipeline
artifact_dir:
  /home/datascience/Jiayuan/NCH/t5_model_artifact:
  - - tokenizer.json
    - .model-ignore
    - config.json
    - generation_config.json
    - score.py
    - special_tokens_map.json
    - tokenizer_config.json
    - pytorch_model.bin
    - runtime.yaml
framework: transformers
model_deployment_id: null
model_id: null

# Summary Status

In [36]:
huggingface_pipeline_model.summary_status()

Actions Needed
Step      Status        Details                                                           
initiate  Done          Initiated the model                                               
prepare() Done          Generated runtime.yaml                                            
                        Generated score.py                                                
                        Serialized model                                                  
                        Populated metadata(Custom, Taxonomy and Provenance)               
verify()  Available     Local tested .predict from score.py                               
save()    Available     Conducted Introspect Test                                         
                        Uploaded artifact to model catalog                                
deploy()  UNKNOWN       Deployed the model                                                
predict() Not Available Called deployment predict endpoint

### verify local model artifact

In [38]:
from t5_model_artifact import score

text = """
If James M. Goodhue could revisit the earth and make a tour among the
daily newspaper offices of St. Paul he would discover that wonderful
strides had been made in the method of producing a newspaper during
the latter half of the past century. Among the first things to attract
the attention of this old-timer would be the web-perfecting press,
capable of producing 25,000 impressions an hour, instead of the old
hand press of 240 impressions an hour; the linotype machine, capable
of setting 6,000 to 10,000 ems per hour, instead of the old hand
compositor producing only 800 to 1,000 ems per hour, and the mailing
machine, enabling one man to do the work of five or six under the
old method. Think of getting out the Sunday Pioneer Press with the
material in use fifty years ago. It would take 600 hand presses, 600
hand pressmen and 600 boys three hours to print the edition, and as
there were no means of stereotyping in those days the forms would have
to be set up 600 times, requiring the services of 5,000 compositors.
Papers printed under these conditions would have to be sold for one
dollar each, and there would not be much profit in it at that. The
first daily papers printed in St. Paul were not conducted or a very
gigantic scale, as the entire force of one office generally consisted
of one pressman, five or six compositors, two editors and a business
manager. A few reminiscences of the trials and tribulations of the
early newspaper manager, editor and compositor may not be wholly
devoid of interest."""

score.predict(text)

{'prediction': [{'summary_text': 'a newspaper would be produced by a web-perfecting press, capable of producing 25,000 impressions an hour, instead of the old hand press of 240 impressions an hour, and the mailing machine, enabling one man to do the work of five or six under the old method. The Sunday Pioneer Press would take 600 hand presses, 600 hand pressmen and 600 boys three hours to print the edition, and as there were no means of stereotyping in those days the forms would have to be set up 600 times, requiring the services of 25,000 impressions an hour, and the linotype, instead of the old hand press of 240 impressions an hour, and the linotype machine, and the mailing machine, and the linotype machine, and the linotype, and the .'}]}

# Register Model

In [40]:
# Register the model
model_id = huggingface_pipeline_model.save(
    display_name="transformer_t5",
    model_version_set="ocid1.datasciencemodelversionset.oc1.iad.amaaaaaawe6j4fqawzqitdpwhetgkxcwmph6rv7jhc7hvk3p4wr3qk6fmcja"   # model version transformer-t5
)

Model is successfully loaded.
['tokenizer.json', '.model-ignore', 'config.json', 'generation_config.json', 'score.py', 'special_tokens_map.json', 'tokenizer_config.json', 'pytorch_model.bin', 'runtime.yaml', 'test_json_output.json']


loop1:   0%|          | 0/4 [00:00<?, ?it/s]

# Deploy Model - DEBUG 

In [54]:
# Deploy and create an endpoint for the huggingface_pipeline_model

huggingface_pipeline_model.deploy(
    display_name="HuggingFace Pipeline Model For Text Summarization",
    deployment_log_group_id="ocid1.loggroup.oc1.iad.amaaaaaawe6j4fqaffwnle4a3vgrxnyr7tknflwlnsuq3tdzcwmmpiu5lxqa",
    deployment_access_log_id="ocid1.log.oc1.iad.amaaaaaawe6j4fqae5vfln65avtsdiv3j47mmpkjmnrarjmopyg3jb57ujfa",
    deployment_predict_log_id="ocid1.log.oc1.iad.amaaaaaawe6j4fqae5vfln65avtsdiv3j47mmpkjmnrarjmopyg3jb57ujfa"
)

print(f"Endpoint: {huggingface_pipeline_model.model_deployment.url}")

loop1:   0%|          | 0/6 [00:00<?, ?it/s]

Endpoint: https://modeldeployment.us-ashburn-1.oci.customer-oci.com/ocid1.datasciencemodeldeployment.oc1.iad.amaaaaaawe6j4fqabusufajddpcyeusvyvzbqnxkazqj7qdkbe2p7al4k5ua


# Prediction using MD Endpoint

In [56]:
# Generate prediction by invoking the deployed endpoint

# The OCI SDK must be installed for this example to function properly.
# Installation instructions can be found here: https://docs.oracle.com/en-us/iaas/Content/API/SDKDocs/pythonsdk.htm

import requests
import oci
from oci.signer import Signer

config = oci.config.from_file(profile_name='jiayuan') # replace with the location of your oci config file
auth = Signer(
  tenancy=config['tenancy'],
  user=config['user'],
  fingerprint=config['fingerprint'],
  private_key_file_location=config['key_file'],
  pass_phrase=config['pass_phrase'])

endpoint = 'https://modeldeployment.us-ashburn-1.oci.customer-oci.com/ocid1.datasciencemodeldeployment.oc1.iad.amaaaaaawe6j4fqabusufajddpcyeusvyvzbqnxkazqj7qdkbe2p7al4k5ua/predict'
body = "Zoom interface is really simple and easy to conduct virtual meetings. It is very easy to share the Zoom link to join the video conference. And overall screen sharing quality is ok. Zoom now claims to have 300 million meeting participants per day. It chose Oracle Corporation co-founded by Larry Ellison and headquartered in Redwood Shores , for its cloud infrastructure deployments over the likes of Amazon, Microsoft, Google, and even IBM to build an enterprise grade experience for its product." # payload goes here
headers = {} # header goes here

requests.post(endpoint, json=body, auth=auth, headers=headers).json()

{'prediction': [{'summary_text': 'Zoom now claims to have 300 million meeting participants per day. It chose Oracle Corporation co-founded by Larry Ellison and headquartered in Redwood Shores .'}]}

In [57]:
pred = requests.post(endpoint, json=body, auth=auth, headers=headers).json()['prediction']
pred[0]['summary_text']

'Zoom now claims to have 300 million meeting participants per day. It chose Oracle Corporation co-founded by Larry Ellison and headquartered in Redwood Shores .'